In [1]:
import cv2
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

2024-01-14 10:06:28.553213: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-14 10:06:28.831201: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 10:06:28.831249: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 10:06:28.832984: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 10:06:28.968305: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-14 10:06:28.969720: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
class_labels = ['A', 'B', 'C', 'D', 'E', 'F']
num_classes = len(class_labels)

In [3]:
img_height, img_width = 224, 224

In [4]:
dataset_path = './../../Training/augmentation1/'

In [5]:
def load_dataset(dataset_path, class_labels):
    images = []
    labels = []
    for label, class_name in enumerate(class_labels):
        class_folder = os.path.join(dataset_path, class_name)
        for image_filename in os.listdir(class_folder):
            image_path = os.path.join(class_folder, image_filename)
            image = cv2.imread(image_path)
            if image is not None:
                image = cv2.resize(image, (img_width, img_height))
                images.append(image)
                labels.append(label)
    return np.array(images), np.array(labels)

In [6]:
images, labels = load_dataset(dataset_path, class_labels)

In [7]:
# Normalize pixel values to be between 0 and 1
images = images / 255.0

# Convert class vectors to binary class matrices
labels = to_categorical(labels, num_classes)

# Splitting dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [8]:
# Model definition
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # Adjust the number of neurons to match the number of classes
])

2024-01-14 10:06:40.081921: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 177209344 exceeds 10% of free system memory.
2024-01-14 10:06:40.255148: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 177209344 exceeds 10% of free system memory.


: 

In [ ]:
# Model compilation
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('./../../new-model.h5', monitor='val_accuracy', save_best_only=True)

In [ ]:
# Training the model
history = model.fit(
    X_train, y_train,
    batch_size=8,
    epochs=30,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, checkpoint]
)